In [12]:
import glob
import os
import shutil
import tensorflow 
from tensorflow.keras import Sequential,models
from tensorflow.keras.layers import Dense,Flatten,BatchNormalization,Conv2D,MaxPooling2D, Dropout, GlobalAveragePooling2D,Activation
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [1]:
# from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.models import Sequential
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint

import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D,Dropout,GlobalAveragePooling2D
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.image import imread
import pathlib
from sklearn.metrics import classification_report, confusion_matrix

# %matplotlib inline

In [2]:
# Unzip training data
from zipfile import ZipFile
file_name = "/content/drive/MyDrive/Colab Notebooks/Computer_Vision/classification/traffic_signal/train_test_data.zip"
with ZipFile(file_name, 'r') as zip:
  zip.extractall('/content/drive/MyDrive/Colab Notebooks/Computer_Vision/classification/traffic_signal/traffic_data')
  print('done')

done


In [7]:
# # Unzip training data
# from zipfile import ZipFile
# file_name = "/content/drive/MyDrive/Colab Notebooks/Computer_Vision/classification/traffic_signal/archive/traffic_Data/TEST.zip"
# with ZipFile(file_name, 'r') as zip:
#   zip.extractall('/content/drive/MyDrive/Colab Notebooks/Computer_Vision/classification/traffic_signal/TEST')
#   print('done')

done


In [3]:
! pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import splitfolders
# Split with a ratio.
# To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
splitfolders.ratio("/content/drive/MyDrive/Colab Notebooks/Computer_Vision/classification/traffic_signal/traffic_data/train_test_data/train",
 output="/content/drive/MyDrive/Colab Notebooks/Computer_Vision/classification/traffic_signal/traffic_data/",
    # seed=1337, 
    ratio=(.8, .2), group_prefix=None, move=False) # default values

Copying files: 3313 files [00:32, 100.47 files/s]


In [7]:
# import splitfolders
# # Split with a ratio.
# # To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
# splitfolders.ratio("/content/drive/MyDrive/Colab Notebooks/Computer_Vision/classification/traffic_signal/archive/traffic_Data/TEST/TEST",
#  output="/content/drive/MyDrive/Colab Notebooks/Computer_Vision/classification/traffic_signal",
#     # seed=1337, 
#     ratio=(.7, .3), group_prefix=None, move=False) # default values

Copying files: 1994 files [18:23,  1.81 files/s]


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
train_dir ='/content/drive/MyDrive/Colab Notebooks/Computer_Vision/classification/traffic_signal/traffic_data/train'
val_dir = '/content/drive/MyDrive/Colab Notebooks/Computer_Vision/classification/traffic_signal/traffic_data/val'
test_dir ='/content/drive/MyDrive/Colab Notebooks/Computer_Vision/classification/traffic_signal/traffic_data/train_test_data/validation'
inf_dir='/content/drive/MyDrive/Colab Notebooks/Computer_Vision/classification/traffic_signal/traffic_data/train_test_data/test'

In [7]:
img_width=150
img_height=150
batch_size=128

In [8]:

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range=30,
                                   zoom_range=0.4,
                                   horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=batch_size,
                                                    # subset="training",
                                                    class_mode='sparse',
                                                    target_size=(img_height, img_width))

Found 2628 images belonging to 58 classes.


In [10]:
test_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range=30,
                                   zoom_range=0.4,
                                   horizontal_flip=True)

test_generator = train_datagen.flow_from_directory(test_dir,
                                                    batch_size=batch_size,
                                                    # subset="training",
                                                    class_mode='sparse',
                                                    target_size=(img_height, img_width))

Found 857 images belonging to 58 classes.


In [9]:
val_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range=30,
                                   zoom_range=0.4,
                                   horizontal_flip=True)

val_generator = train_datagen.flow_from_directory(val_dir,
                                                    batch_size=batch_size,
                                                    # subset="training",
                                                    class_mode='sparse',
                                                    target_size=(img_height, img_width))

Found 685 images belonging to 58 classes.


Ceating VGG16

In [14]:
img_shape = 150
conv_base = VGG16(
    weights='imagenet',
    include_top = False,
    input_shape=(img_shape,img_shape,3)
)

58889256/58889256 [==============================] - 4s 0us/step


In [15]:
from tensorflow.keras.layers import BatchNormalization

In [16]:

model = Sequential()

model.add(conv_base)
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(58,activation='softmax'))

In [17]:
model.compile(optimizer='adam',loss = "sparse_categorical_crossentropy",metrics=['accuracy'])

In [18]:
from tensorflow.keras.callbacks import ModelCheckpoint
filepath="/kaggle/working/VGG16_150-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
history = model.fit(train_generator,epochs=40,validation_data=val_generator,callbacks=callbacks_list)

Epoch 1/40
21/21 [==============================] - ETA: 0s - loss: 4.8818 - accuracy: 0.0304
Epoch 1: val_accuracy improved from -inf to 0.02336, saving model to /kaggle/working/VGG16_150-01-0.02.hdf5
21/21 [==============================] - 62s 2s/step - loss: 4.8818 - accuracy: 0.0304 - val_loss: 148.1149 - val_accuracy: 0.0234
Epoch 2/40
21/21 [==============================] - ETA: 0s - loss: 4.3195 - accuracy: 0.0788
Epoch 2: val_accuracy did not improve from 0.02336
21/21 [==============================] - 29s 1s/step - loss: 4.3195 - accuracy: 0.0788 - val_loss: 681.3167 - val_accuracy: 0.0044
Epoch 3/40
21/21 [==============================] - ETA: 0s - loss: 4.0271 - accuracy: 0.1168
Epoch 3: val_accuracy did not improve from 0.02336
21/21 [==============================] - 28s 1s/step - loss: 4.0271 - accuracy: 0.1168 - val_loss: 45.9064 - val_accuracy: 0.0234
Epoch 4/40
21/21 [==============================] - ETA: 0s - loss: 3.8177 - accuracy: 0.1461
Epoch 4: val_accuracy 